# 第五章 广度优先算法


广度优先和深度优先遍历相似，都是查找方法。它也是从某个状态出发去查询所有可以到达的状态，但是和深度优先不同，广度优先总是先查询距离初始状态最近的状态。

本章主要涉及的问题有如下几个：
- 选课的智慧：如何确定选课的顺序
- 寻找制高点：寻找制高点，强占有利地形
- 合法的括号：从非合法括号序列中寻找合法括号序列
- 树的右侧：从树的侧边观察树的伟岸身躯

## 5.1 什么是广度优先遍历

对比第四章中的DFS，广度优先算法在搜索中，采用由近及远的方式。先访问距离起始点最近的点，再访问远一点的点。比如访问距离初始点只有一步的点，然后再访问距离初始点两步的点。

因此广度优先算法也叫层次遍历法，一层一层地去寻找答案。

## 5.2 选课的智慧

选课的时候，我们常常遇到某些课程需要修过前置课程，然后才可以进行选课。我们可以用一个二维数组去储存这种关系，如果第i个课程是第j个课程的先修课，那么数组中的(i+1,j+1)位置是1。

### 5.2.1 广度优先遍历

比如说小明有法律问题需要咨询，首先我们想到的是自己的朋友，看看自己朋友（一度人脉）里面有没有律师。如果自己朋友里面没有律师，我们会去想朋友的朋友（二度人脉）里面是否有律师，等等。这种检索方式就是广度优先遍历。

每段时间，我们有一个考察对象，去询问是否是律师。这边需要用到的数据结构叫做队列，它的性质是先入先出，后入后出。比如排队买咖啡，作为有素质的人，必然会排到队伍的尾端。先来的人先买，买完离开，后来的人加入队尾。

### 5.2.2 问题求解

首先需要建立一个数组去储存每门课的先修数量，讲每门课的先修数量初始化为0，课程数量为numCoureses。

In [21]:
preList = [[0,0,1,0,0],[0,0,1,0,0],[0,0,0,0,1],[0,0,0,0,1],[0,0,0,0,0]]
numCourses = len(preList)
preListCount = [0]*numCourses

[0, 0, 0, 0, 0]

接下来，需要通过先修课的二维数组preList来计算每门课的先修课数量，代码如下：

In [22]:
for line in preList:               # 取出二维数组的每一行
    for i in range(len(line)):     # 针对每一行计算和，对preListCount进行赋值
        if line[i] == 1:
            preListCount[i] += 1

接下来，建立一个队列canTake去储存目前可以选择的课程。讲那些先修课数量为0的课程加入队列canTake。

In [41]:
canTake = []
for i in range(len(preListCount)):
    if preListCount[i] == 0:
        canTake.append(i)

经过以上步骤，我们建立了存放每门课程先修课数量的数组preListCount和存放目前可以进行选修的课的队列canTake。接下来，就可以使用广度优先算法遍历，进行选课了。先修课数量，相当于距离决策的距离。可以选修的课队列相当于目前的搜索范围，先入先出，当每一门课被选中的时候，更新先修课数量的数组，实际上更新了距离决策的距离。最后我们加上限定条件，结束BFS，就完成了选课。

In [43]:
classTake = []
while len(canTake) != 0:
    thisClass = canTake[0]
    del canTake[0]
    classTake.append(thisClass)
    print(classTake)
    for i in range(numCourses):
        if preList[thisClass][i] == 1:
            preListCount[i] -= 1
            if preListCount[i] == 0:
                canTake.append(i)

[0]
[0, 1]
[0, 1, 3]


### 5.2.3 最终代码

In [49]:
def bfs(numCourses,preList):
    preListCount = [0]*numCourses
    for line in preList:
        for i in range(len(line)):
            if line[i] == 1:
                preListCount += 1
    canTake = []
    for i in range(len(preListCount)):
        if preListCount[i] == 0:
            canTake.append(i)
    classTake = []
    while len(canTake) != 0:
        thisClass = canTake[0]
        del canTake[0]
        classTake.append(thisClass)
        print(classTake)
        for i in range(numCourses):
            if preList[thisClass][i] == 1:
                preListCount[i] -= 1
                if preListCount[i] == 0:
                    canTake.append(i)

## 5.3 寻找制高点

首先吧地图网格化，每个格点用一个数字代表高度，之后就可以用二维数组表示海拔。制高点的定义是，可以一直向下到达东南西北四个方向的点。在向下走的过程中是不能够爬坡的，也就是说，只能向比当前点低的点或者一样高的点的方向去走，最终能够到达地图的四个边缘。

### 5.3.1 问题求解

这个问题是典型的搜索问题，最暴力的想法是遍历每个点，查看每个点是否能到达四个边缘。但是，不像迷宫问题，搜索的目标点不是一个单点，而是所有的边缘点，这种暴力搜索显然是效率很低的。

使用逆向思维，我们从边缘的所有点出发，向内部开始搜索，查看下一个节点的高度是否高于或者等于自身的高度。然后标记那些能够达到的点为True,继续搜索下去，直到不能走为止。按照同样的思路分别标记从四个边缘出发可以到达的点，那么最终四者均为True的点，就是我们寻找的制高点。

首先，我们有的是一张地图，可以用二维数组去表示。代码框架如下：

In [58]:
matrix = [[1,3,2,3,5],
          [3,4,5,6,3],
          [2,7,4,3,3],
          [5,2,2,3,1]]

In [59]:
def solve(matrix):    # matrix是储存地图的二维数组
    if not matrix: 
        return matrix # 如果二维数组为空，那么返回空
    pass              # 之后的逻辑，用pass代替

为了更好地表示移动方向，同样使用一个二维数组来储存上下左右四个方向。这个技巧会在以后的问题中多次使用到。

In [60]:
direction = [[0,1],[0,-1],[1,0],[-1,0]]  # 数组中的四个元素代表四个方向

计算地图的大小

In [61]:
m = len(matrix)    # 计算地图有多少行
n = len(matrix[0]) # 计算地图有多少列

接下来，从山的背面上山为例，来寻找制高点，队列是完成广度优先遍历必备的数据结构。所以，先把第一行的点全部加入队列，然后就可以开始广度优先遍历了：

In [69]:
toppoint = [(0,y) for y in range(n)]

由于从上下左右四个方向上山都是同一个子问题，因此最好把广度优先遍历写成一个函数，代码模板如下：

In [72]:
def bfs(set,m,n,matrix):
    while len(set) > 0:
        x,y = set.pop(0)
        # 一些逻辑处理
        pass
        if True : # 可以到达
            set.append((x,y))
            

广度优先遍历算法的步骤是从队头取出一个点，然后从改点出发，通过方向数组建立新的点，查询这些点是否可以到达。记录到达的点，然后把这些点加入队列。为了保存那些可以到达的点，需要在广度优先遍历之前，建立一个队列的副本。在Python中，list()函数可以把元组转换成列表，通过list()函数建立set的一个副本，作为新的队列。

In [77]:
def bfs(set,m,n,matrix):
    queue = list(set)
    while len(queue) > 0:
        x,y = queue.pop(0) # 取出队列中的第一个元素，并删除，通过两个变量来接受横纵坐标
        for d in direction:
            nx = x + d[0]
            ny = y + d[1]
            if 0<= nx and nx < m and 0<=ny and ny< n:
                if matrix[nx][ny] >= matrix[x][y]:
                    if (nx,ny) not in set:
                        queue.append((nx,ny))
                        set.add((nx,ny))

经过上面的广度优先遍历，可以得到从某一个方向护法可以到达的点，那么我们执行四次，就可以得到四个方向的结果了。为了后面的集合运算，我们使用set()函数来把列表转化成集合：

In [78]:
topPoint = set((0,y) for y in range(n))
leftPoint = set((x,0) for x in range(m))
bottomPoint = set((m-1,y) for y in range(n))
rightPoint = set((x,n-1) for x in range(m))
bfs(topPoint, m, n, matrix)
bfs(leftPoint, m, n, matrix)
bfs(bottomPoint, m, n, matrix)
bfs(rightPoint, m, n, matrix)

In [83]:
result = topPoint & bottomPoint & leftPoint & rightPoint

### 5.2.3 最终代码



In [84]:
def bfs(set,m,n,matrix):
    direction = [[0,1],[0,-1],[1,0],[-1,0]]
    queue = list(set)
    while len(queue) > 0:
        x,y = queue.pop(0) # 取出队列中的第一个元素，并删除，通过两个变量来接受横纵坐标
        for d in direction:
            nx = x + d[0]
            ny = y + d[1]
            if 0<= nx and nx < m and 0<=ny and ny< n:
                if matrix[nx][ny] >= matrix[x][y]:
                    if (nx,ny) not in set:
                        queue.append((nx,ny))
                        set.add((nx,ny))

def solve(matrix):
    if not matrix:
        return matrix
    m = len(matrix)
    n = len(matrix[0])
    topPoint = set((0,y) for y in range(n))
    leftPoint = set((x,0) for x in range(m))
    bottomPoint = set((m-1,y) for y in range(n))
    rightPoint = set((x,n-1) for x in range(m))
    bfs(topPoint, m, n, matrix)
    bfs(leftPoint, m, n, matrix)
    bfs(bottomPoint, m, n, matrix)
    bfs(rightPoint, m, n, matrix)
    result = topPoint & bottomPoint & leftPoint & rightPoint
    return result

matrix = [[1,3,2,3,5],
          [3,4,5,6,3],
          [2,7,4,3,3],
          [5,2,2,3,1]]
s = solve(matrix)
print(s)    

{(1, 2), (1, 3), (2, 1)}


## 5.4 合法的括号

